# 🚀 Day 1: Memory Coalescing - The Key to GPU Performance

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-02/day-1-memory-coalescing.ipynb)

## Learning Philosophy

> **CUDA C++ First, Python/Numba as Optional Backup**

This notebook shows:
1. **CUDA C++ code** - The PRIMARY implementation you should learn
2. **Python/Numba code** - OPTIONAL for quick interactive testing in Colab

---

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
# Python/Numba is OPTIONAL - for quick interactive testing only
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math
import time

print("\n⚠️  Remember: CUDA C++ code is the PRIMARY learning material!")

# Day 1: Memory Coalescing - The Key to GPU Performance

Memory bandwidth is the #1 bottleneck in most GPU programs. Today you'll learn:
- How GPUs access global memory
- What memory coalescing means
- How to write coalesced access patterns
- Measuring the performance impact

---

## 1. How GPU Memory Access Works

When a warp (32 threads) accesses global memory, the hardware:
1. Collects all memory addresses from all threads
2. Groups them into **memory transactions** (32, 64, or 128 bytes)
3. Fetches data in as few transactions as possible

### CUDA C++ Example: Coalesced vs Non-Coalesced

```cpp
// coalescing_demo.cu
#include <stdio.h>
#include <cuda_runtime.h>

// GOOD: Coalesced access - adjacent threads access adjacent memory
__global__ void coalescedCopy(const float* src, float* dst, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        dst[idx] = src[idx];  // Thread 0→addr 0, Thread 1→addr 1, ...
    }
}

// BAD: Strided access - adjacent threads access scattered memory
__global__ void stridedCopy(const float* src, float* dst, int n, int stride) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int strided_idx = idx * stride;  // Thread 0→addr 0, Thread 1→addr 32, ...
    if (strided_idx < n) {
        dst[strided_idx] = src[strided_idx];
    }
}
```

```
COALESCED ACCESS (Good):
Thread:    0    1    2    3   ...  31
Address:  [0]  [1]  [2]  [3] ... [31]
           └────────────────────────┘
                ONE 128-byte transaction!

NON-COALESCED ACCESS (Bad):
Thread:    0    1    2    3   ...  31
Address:  [0] [32] [64] [96] ...[992]
           │    │    │    │       │
           └────┴────┴────┴───────┘
           32 separate transactions! (32x slower)
```

### Python/Numba (Optional - Interactive Testing)

In [ ]:
import numpy as np
from numba import cuda
import math
import time

print("CUDA device:", cuda.get_current_device().name.decode())

# Get memory bandwidth info
device = cuda.get_current_device()
print(f"Warp size: {device.WARP_SIZE}")
print(f"Max threads per block: {device.MAX_THREADS_PER_BLOCK}")

## 2. The Coalescing Rule

### The Golden Rule:
**Adjacent threads (within a warp) should access adjacent memory locations.**

### Transaction Sizes:
- 32 bytes (8 × float32)
- 64 bytes (16 × float32)
- 128 bytes (32 × float32) ← ideal for a warp!

### Good vs Bad Patterns:

| Pattern | Example | Coalesced? |
|---------|---------|------------|
| Sequential | `arr[threadIdx.x]` | ✅ Yes |
| Strided | `arr[threadIdx.x * stride]` | ❌ No (if stride > 1) |
| Random | `arr[random_index]` | ❌ No |
| Row-major 2D | `arr[row][col]` with col = threadIdx.x | ✅ Yes |
| Column-major 2D | `arr[row][col]` with row = threadIdx.x | ❌ No |

## 3. Demonstrating Coalesced vs Non-Coalesced Access

Let's create two kernels that do the same work but with different access patterns:

In [ ]:
@cuda.jit
def coalesced_copy(src, dst, n):
    """
    COALESCED: Adjacent threads access adjacent elements.
    Thread 0 → src[0], Thread 1 → src[1], ...
    """
    idx = cuda.grid(1)
    if idx < n:
        dst[idx] = src[idx]

@cuda.jit
def strided_copy(src, dst, n, stride):
    """
    NON-COALESCED: Threads access memory with stride.
    Thread 0 → src[0], Thread 1 → src[stride], Thread 2 → src[2*stride]...
    """
    tid = cuda.grid(1)
    if tid < n // stride:
        # Calculate strided index
        idx = tid * stride
        if idx < n:
            dst[idx] = src[idx]

def benchmark_access_pattern(n, stride=1, iterations=100):
    """Benchmark different access patterns"""
    src = cuda.to_device(np.random.randn(n).astype(np.float32))
    dst = cuda.device_array(n, dtype=np.float32)
    
    threads = 256
    blocks = math.ceil(n / threads)
    
    # Warmup
    if stride == 1:
        coalesced_copy[blocks, threads](src, dst, n)
    else:
        strided_copy[blocks, threads](src, dst, n, stride)
    cuda.synchronize()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        if stride == 1:
            coalesced_copy[blocks, threads](src, dst, n)
        else:
            strided_copy[blocks, threads](src, dst, n, stride)
    cuda.synchronize()
    elapsed = (time.perf_counter() - start) / iterations
    
    # Calculate bandwidth
    bytes_transferred = 2 * n * 4  # Read + write, float32
    bandwidth = bytes_transferred / elapsed / 1e9
    
    return elapsed * 1000, bandwidth

# Compare patterns
n = 100_000_000  # 100M elements

print(f"Array size: {n:,} elements ({n * 4 / 1e9:.2f} GB)")
print("=" * 60)
print(f"{'Pattern':<25} | {'Time (ms)':<12} | {'Bandwidth (GB/s)'}")
print("-" * 60)

time_coal, bw_coal = benchmark_access_pattern(n, stride=1)
print(f"{'Coalesced (stride=1)':<25} | {time_coal:<12.3f} | {bw_coal:.1f}")

for stride in [2, 4, 8, 16, 32]:
    time_s, bw_s = benchmark_access_pattern(n, stride=stride)
    slowdown = time_s / time_coal
    print(f"{'Strided (stride=' + str(stride) + ')':<25} | {time_s:<12.3f} | {bw_s:.1f} ({slowdown:.1f}x slower)")

## 4. 2D Array Access Patterns

For 2D arrays (matrices, images), access pattern matters even more!

```
Row-Major Layout (C/NumPy default):
┌─────────────────────────────────┐
│ (0,0) (0,1) (0,2) (0,3) ... │ Row 0 (contiguous in memory)
│ (1,0) (1,1) (1,2) (1,3) ... │ Row 1
│ (2,0) (2,1) (2,2) (2,3) ... │ Row 2
└─────────────────────────────────┘

Memory: [0,0][0,1][0,2][0,3]...[1,0][1,1][1,2]...

✅ COALESCED: Threads read along a row (vary column)
   Thread 0 → (row, 0), Thread 1 → (row, 1), ...

❌ NON-COALESCED: Threads read down a column (vary row)
   Thread 0 → (0, col), Thread 1 → (1, col), ...
```

In [ ]:
@cuda.jit
def row_major_read(matrix, output, rows, cols):
    """
    COALESCED: Each warp reads along a row.
    threadIdx.x corresponds to column (fast-changing dimension)
    """
    col, row = cuda.grid(2)
    
    if row < rows and col < cols:
        # Reading matrix[row][col] - threads in a warp read adjacent columns
        output[row, col] = matrix[row, col] * 2.0

@cuda.jit
def col_major_read(matrix, output, rows, cols):
    """
    NON-COALESCED: Each warp reads down a column.
    threadIdx.x corresponds to row (slow-changing dimension)
    """
    row, col = cuda.grid(2)  # Note: swapped!
    
    if row < rows and col < cols:
        # Same operation, but different thread mapping
        output[row, col] = matrix[row, col] * 2.0

def benchmark_2d_pattern(rows, cols, pattern='row', iterations=100):
    """Benchmark 2D access patterns"""
    matrix = cuda.to_device(np.random.randn(rows, cols).astype(np.float32))
    output = cuda.device_array((rows, cols), dtype=np.float32)
    
    threads = (16, 16)  # 256 threads per block
    
    if pattern == 'row':
        blocks = (math.ceil(cols / 16), math.ceil(rows / 16))
        kernel = row_major_read
    else:
        blocks = (math.ceil(rows / 16), math.ceil(cols / 16))
        kernel = col_major_read
    
    # Warmup
    kernel[blocks, threads](matrix, output, rows, cols)
    cuda.synchronize()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        kernel[blocks, threads](matrix, output, rows, cols)
    cuda.synchronize()
    elapsed = (time.perf_counter() - start) / iterations
    
    bytes_transferred = 2 * rows * cols * 4
    bandwidth = bytes_transferred / elapsed / 1e9
    
    return elapsed * 1000, bandwidth

# Benchmark 2D patterns
rows, cols = 4096, 4096  # 64MB matrix

print(f"Matrix size: {rows} × {cols} ({rows * cols * 4 / 1e6:.1f} MB)")
print("=" * 55)

time_row, bw_row = benchmark_2d_pattern(rows, cols, 'row')
time_col, bw_col = benchmark_2d_pattern(rows, cols, 'col')

print(f"Row-major (coalesced):     {time_row:.3f} ms, {bw_row:.1f} GB/s")
print(f"Column-major (strided):    {time_col:.3f} ms, {bw_col:.1f} GB/s")
print(f"\nSpeedup from coalescing: {time_col/time_row:.2f}x")

## 5. Matrix Transpose: A Classic Coalescing Problem

Matrix transpose is tricky because:
- Reading rows (coalesced) means writing columns (non-coalesced)
- Reading columns (non-coalesced) means writing rows (coalesced)

You can't win with naive approach! (We'll fix this with shared memory in Day 2)

In [ ]:
@cuda.jit
def transpose_naive(input_matrix, output_matrix, rows, cols):
    """
    Naive transpose: coalesced reads, non-coalesced writes
    """
    col, row = cuda.grid(2)
    
    if row < rows and col < cols:
        # Read from input[row][col] (coalesced - threads read along row)
        # Write to output[col][row] (non-coalesced - threads write to scattered locations)
        output_matrix[col, row] = input_matrix[row, col]

@cuda.jit
def transpose_read_coalesced(input_matrix, output_matrix, rows, cols):
    """
    Same as naive - prioritize coalesced reads
    """
    col, row = cuda.grid(2)
    
    if row < rows and col < cols:
        output_matrix[col, row] = input_matrix[row, col]

@cuda.jit
def transpose_write_coalesced(input_matrix, output_matrix, rows, cols):
    """
    Prioritize coalesced writes (non-coalesced reads)
    """
    col, row = cuda.grid(2)
    
    if row < rows and col < cols:
        # Read from input[col][row] (non-coalesced - scattered reads)
        # Write to output[row][col] (coalesced - threads write along row)
        output_matrix[row, col] = input_matrix[col, row]

def benchmark_transpose(rows, cols, kernel, iterations=100):
    """Benchmark transpose kernel"""
    input_mat = cuda.to_device(np.random.randn(rows, cols).astype(np.float32))
    output_mat = cuda.device_array((cols, rows), dtype=np.float32)
    
    threads = (16, 16)
    blocks = (math.ceil(cols / 16), math.ceil(rows / 16))
    
    # Warmup
    kernel[blocks, threads](input_mat, output_mat, rows, cols)
    cuda.synchronize()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        kernel[blocks, threads](input_mat, output_mat, rows, cols)
    cuda.synchronize()
    elapsed = (time.perf_counter() - start) / iterations
    
    bytes_transferred = 2 * rows * cols * 4
    bandwidth = bytes_transferred / elapsed / 1e9
    
    return elapsed * 1000, bandwidth

# Benchmark transpose patterns
rows, cols = 4096, 4096

print(f"Matrix transpose: {rows} × {cols}")
print("=" * 55)

time_read, bw_read = benchmark_transpose(rows, cols, transpose_read_coalesced)
time_write, bw_write = benchmark_transpose(rows, cols, transpose_write_coalesced)

print(f"Coalesced reads:   {time_read:.3f} ms, {bw_read:.1f} GB/s")
print(f"Coalesced writes:  {time_write:.3f} ms, {bw_write:.1f} GB/s")
print(f"\n💡 Neither is optimal! We need shared memory (Day 2) to fix this.")

## 🎯 Exercises

### Exercise 1: Identify the Access Pattern

For each kernel below, identify if the access is coalesced or not:

In [ ]:
# Exercise 1: Identify coalesced vs non-coalesced

# Pattern A
@cuda.jit
def pattern_a(arr, n):
    idx = cuda.grid(1)
    if idx < n:
        arr[idx] = idx  # TODO: Coalesced or not?

# Pattern B
@cuda.jit
def pattern_b(arr, n):
    idx = cuda.grid(1)
    if idx < n:
        arr[n - 1 - idx] = idx  # TODO: Coalesced or not?

# Pattern C
@cuda.jit
def pattern_c(arr, n):
    idx = cuda.grid(1)
    if idx < n // 2:
        arr[idx * 2] = idx  # TODO: Coalesced or not?

# Pattern D
@cuda.jit
def pattern_d(matrix, rows, cols):
    col, row = cuda.grid(2)
    if row < rows and col < cols:
        matrix[row, col] = row + col  # TODO: Coalesced or not?

print("Analyze each pattern and answer:")
print("Pattern A: ?")
print("Pattern B: ?")
print("Pattern C: ?")
print("Pattern D: ?")

# Answers:
# A: Coalesced (sequential access)
# B: Coalesced (reverse sequential is still contiguous within warp)
# C: Non-coalesced (stride of 2)
# D: Coalesced (threadIdx.x maps to col, which is the fast dimension)

### Exercise 2: Fix the Non-Coalesced Access

The kernel below processes a 2D array but has non-coalesced access. Fix it!

In [ ]:
# Exercise 2: Fix the access pattern

@cuda.jit
def process_matrix_bad(matrix, output, rows, cols):
    """BAD: Non-coalesced access"""
    # Problem: threadIdx.x maps to row, causing non-coalesced column access
    row, col = cuda.grid(2)  # This mapping is wrong!
    
    if row < rows and col < cols:
        output[row, col] = matrix[row, col] * 2.0

@cuda.jit
def process_matrix_good(matrix, output, rows, cols):
    """TODO: Fix to be coalesced"""
    # TODO: Change the thread-to-index mapping
    row, col = cuda.grid(2)  # FIX THIS LINE
    
    if row < rows and col < cols:
        output[row, col] = matrix[row, col] * 2.0

# Test your fix
# rows, cols = 2048, 2048
# ... benchmark both versions

## 📝 Key Takeaways

### Memory Coalescing Rules:

1. **Adjacent threads should access adjacent memory**
   - Thread 0 → mem[0], Thread 1 → mem[1], etc.

2. **For 2D arrays in row-major (C/NumPy):**
   - `threadIdx.x` should map to the column index
   - `col, row = cuda.grid(2)` is the correct order

3. **Strided access kills performance**
   - Stride of 2 → ~50% efficiency
   - Stride of 32 → ~3% efficiency

4. **Some patterns can't be fixed with coalescing alone**
   - Matrix transpose needs shared memory
   - We'll learn this tomorrow!

### Performance Impact:
- Coalesced: 200-400 GB/s on T4
- Non-coalesced: 10-50 GB/s
- **10-40x performance difference!**

---

### 📚 Next Up: Day 2 - Shared Memory
- Using shared memory for tile-based algorithms
- Fixing the transpose problem
- Thread synchronization with `__syncthreads()`

---

### 🔗 Resources
- [Device Memory Access](../../cuda-programming-guide/03-advanced/device-memory-access.md)
- [Performance Optimization](../../cuda-programming-guide/03-advanced/performance-optimization.md)